In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from re import search
from time import sleep
import random


startingUrl = "https://vancouver.craigslist.org/search/apa?query=ubc&min_price=&max_price=&availabilityMode=0&sale_date=all+dates"
# add 's=START_NUMBER' to the query in order to scrape through paginator

# try to avoid scraper defense
HEADER = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
req = requests.get(startingUrl, HEADER)
soup = BeautifulSoup(req.content, 'html.parser')
# find all rows
resultRows = soup.find_all("li", {"class": "result-row"})

data = []
for result in resultRows:
    row = {}
    time = result.find("time", {"class": "result-date"})['datetime']
    header = result.find("h3", {"class": "result-heading"}).find("a")
    name = header.text
    href = header['href']
    price = result.find("span", {"class": "result-price"}).text
    bdr = result.find("span", {"class": "housing"})
    if bdr:
        bdr = bdr.text
        if search("br", bdr):
            bdr = int(bdr[bdr.find(' ')+len(' '):bdr.rfind('br')])
        else:
            bdr = 1
    else:
        bdr = 1
    # put everything together
    row['name'] = name
    row['href'] = href
    row['time'] = time
    row['price'] = price
    row['bedroom'] = bdr
    data.append(row)
# Put everything together in a dataframe
df = pd.DataFrame(data)



In [ ]:
# Pagination Handler
totalCounts = int(soup.find("span", {"class" : "totalcount"}).text)
totalCounts
def getPaginatorParameter(df, totalCounts):
    result = []
    temp = len(df)
    one_iter = len(df)
    while temp < totalCounts:
        result.append(temp)
        temp += one_iter
    return result
parameters = getPaginatorParameter(df, totalCounts)



for param in parameters:
    link = startingUrl + '&s=' + str(param)
    req = requests.get(link, HEADER)
    soup = BeautifulSoup(req.content, 'html.parser')
# find all rows
    resultRows = soup.find_all("li", {"class": "result-row"})
    tempData = []
    for result in resultRows:
        row = {}
        time = result.find("time", {"class": "result-date"})['datetime']
        header = result.find("h3", {"class": "result-heading"}).find("a")
        name = header.text
        href = header['href']
        price = result.find("span", {"class": "result-price"}).text
        bdr = result.find("span", {"class": "housing"})
        if bdr:
            bdr = bdr.text
            if search("br", bdr):
                bdr = int(bdr[bdr.find(' ')+len(' '):bdr.rfind('br')])
            else:
                bdr = 1
        else:
            bdr = 1
        # put everything together
        row['name'] = name
        row['href'] = href
        row['time'] = time
        row['price'] = price
        row['bedroom'] = bdr
        tempData.append(row)
    # Put everything together in a dataframe
    tempDf = pd.DataFrame(tempData)
    df = df.append(tempDf, ignore_index=True)

In [ ]:
# Message the data
# Convert string to datetime
df['time'] = pd.to_datetime(df['time'])
# Currency
df['price'] = df['price'].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(np.int64)

df.drop(df.loc[df['price']==0].index, inplace=True)
df = df.drop_duplicates(subset=['name','price','bedroom'], keep='first')
df

In [ ]:
# location = []
# for link in df['href']:
#     req = requests.get(link, HEADER)
#     print(link)
#     soup = BeautifulSoup(req.content, 'html.parser')
#     result = soup.find("div", {"class": "mapaddress"})
#     # If listing does specifies map address
#     if result.text:
#         location.append(result.text)
#     # If listing does not specify map address
#     else:
#         location.append('location not found')

hrefs = df['href'].tolist()
location = []
for index in range(0,len(hrefs)):
    req = requests.get(hrefs[index], headers={'User-Agent': 'Custom'})
    
    sleep(random.randint(3, 7))
    print("currently on row {}".format(index + 1))
    if req.status_code == 200:
        soup = BeautifulSoup(req.content, 'html.parser')
        result = soup.find("div", {"class": "mapaddress"})
        # If listing does specifies map address
        if result is not None:
            location.append(result.text)
        # If listing does not specify map address
        else:
            location.append('location not found')
    else:
        print("unexpected status code {}".format(req.status_code))
        break
location



In [18]:
df['location'] = location
df.to_csv('data_frame.csv', index = False)
